In [94]:
import numpy as np 
import pandas as pd

from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [95]:
url = '../data/default_nyc_taxi.csv'
data = pd.read_csv(url, parse_dates=['timestamp'], index_col='timestamp')

data = data[["value"]]
data.head()

,value
timestamp,
2014-07-01 00:00:00,10844
2014-07-01 00:30:00,8127
2014-07-01 01:00:00,6210
2014-07-01 01:30:00,4656
2014-07-01 02:00:00,3820


In [96]:
period = 48
print(data.index[:period])

DatetimeIndex(['2014-07-01 00:00:00', '2014-07-01 00:30:00',
               '2014-07-01 01:00:00', '2014-07-01 01:30:00',
               '2014-07-01 02:00:00', '2014-07-01 02:30:00',
               '2014-07-01 03:00:00', '2014-07-01 03:30:00',
               '2014-07-01 04:00:00', '2014-07-01 04:30:00',
               '2014-07-01 05:00:00', '2014-07-01 05:30:00',
               '2014-07-01 06:00:00', '2014-07-01 06:30:00',
               '2014-07-01 07:00:00', '2014-07-01 07:30:00',
               '2014-07-01 08:00:00', '2014-07-01 08:30:00',
               '2014-07-01 09:00:00', '2014-07-01 09:30:00',
               '2014-07-01 10:00:00', '2014-07-01 10:30:00',
               '2014-07-01 11:00:00', '2014-07-01 11:30:00',
               '2014-07-01 12:00:00', '2014-07-01 12:30:00',
               '2014-07-01 13:00:00', '2014-07-01 13:30:00',
               '2014-07-01 14:00:00', '2014-07-01 14:30:00',
               '2014-07-01 15:00:00', '2014-07-01 15:30:00',
               '2014-07-

In [97]:
print('From  ' + str(np.min(data.index)) + '  to  ' +str(np.max(data.index)))

From  2014-07-01 00:00:00  to  2015-01-31 23:30:00


In [98]:
print('Data size: %d \nNumber of data per day: %d \nNumber of days: %d' %(data.shape[0], period, data.shape[0] / period))

Data size: 10320 
Number of data per day: 48 
Number of days: 215


In [99]:
print('Missing value: ', data.isnull().to_numpy().sum())

Missing value:  0


In [100]:
ratio = 0.55
train_size = int(data.shape[0] * ratio)
train = data[:train_size]
test = data[train_size:]

dates_train = data.index[:train_size]
dates_test = data.index[train_size:]

print('Train size: ', ratio)

print('\n\nTRAIN SET:  from  ' + str(np.min(dates_train)) + '  to  ' +str(np.max(dates_train)))
print('Data size: ', train.shape[0])
print('Number of days: ', int(train.shape[0] / period))


print('\n\nTEST SET:  from  ' + str(np.min(dates_test)) + '  to  ' +str(np.max(dates_test)))
print('Data size: ', test.shape[0])
print('Number of days: ', int(test.shape[0] / period))

Train size:  0.55


TRAIN SET:  from  2014-07-01 00:00:00  to  2014-10-27 05:30:00
Data size:  5676
Number of days:  118


TEST SET:  from  2014-10-27 06:00:00  to  2015-01-31 23:30:00
Data size:  4644
Number of days:  96


In [101]:
scaler = StandardScaler()
scaler = scaler.fit(train.values.reshape(-1, 1))

train = scaler.transform(train.values.reshape(-1, 1))
test = scaler.transform(test.values.reshape(-1, 1))

In [102]:
def split_into_windows(X, length):
    windows = []
    for start in range(0, len(X) - length + 1):
        windows.append(X[start: start + length])
    return np.array(windows)

In [103]:
window_length = 48

X_train = split_into_windows(train, window_length)
X_test = split_into_windows(test, window_length)

_, input_length, input_dim = X_train.shape

In [104]:
X_train.shape

(5629, 48, 1)

In [105]:
X_train[0].shape

(48, 1)

In [106]:
# params
dim_hidden1 = 24 # 1/2 the length of one window
dim_hidden2 = 12 # 1/4 the length of one window


# model
model = Sequential()
model.add(LSTM(units=dim_hidden1, activation='relu', input_shape=(input_length, input_dim), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=dim_hidden2, activation='relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=dim_hidden2, activation='relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=dim_hidden1, activation='relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(1))

# summary
model.compile(loss='mse', optimizer='adam')
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 48, 24)            2496      
                                                                 
 dropout_12 (Dropout)        (None, 48, 24)            0         
                                                                 
 lstm_13 (LSTM)              (None, 48, 12)            1776      
                                                                 
 dropout_13 (Dropout)        (None, 48, 12)            0         
                                                                 
 lstm_14 (LSTM)              (None, 48, 12)            1200      
                                                                 
 dropout_14 (Dropout)        (None, 48, 12)            0         
                                                                 
 lstm_15 (LSTM)              (None, 48, 24)           

In [107]:
# params
batch_size = 48 # 24 hours == length of the window
epochs = 50
validation_split = 0.1
es = EarlyStopping(monitor='val_loss', patience=5)


# fitting
history = model.fit(X_train, X_train, 
                batch_size = batch_size, 
                epochs = epochs, 
                validation_split = validation_split, 
                callbacks = [es])

Epoch 1/50


106/106 [==============================] - 7s 33ms/step - loss: 1.4967 - val_loss: 1.1093
Epoch 2/50
106/106 [==============================] - 3s 30ms/step - loss: 0.8124 - val_loss: 0.5810
Epoch 3/50
106/106 [==============================] - 3s 30ms/step - loss: 0.4322 - val_loss: 0.3273
Epoch 4/50
106/106 [==============================] - 3s 29ms/step - loss: 0.3147 - val_loss: 0.2446
Epoch 5/50
106/106 [==============================] - 3s 29ms/step - loss: 0.2480 - val_loss: 0.1816
Epoch 6/50
106/106 [==============================] - 3s 30ms/step - loss: 0.2071 - val_loss: 0.1418
Epoch 7/50
106/106 [==============================] - 3s 29ms/step - loss: 0.1750 - val_loss: 0.1105
Epoch 8/50
106/106 [==============================] - 3s 29ms/step - loss: 0.1515 - val_loss: 0.0891
Epoch 9/50
106/106 [==============================] - 3s 31ms/step - loss: 0.1340 - val_loss: 0.0769
Epoch 10/50
106/106 [==============================] - 4s 34ms/step - loss: 0.1204 - val_loss: 0.0696


In [108]:
pred_x_test = model.predict(X_test)
pred_x_train = model.predict(X_train)

176/176 [==============================] - 1s 8ms/step


In [109]:
mse = lambda x, x_pred: np.mean(np.abs(x - x_pred)**2, axis=1)

train_mae_loss = pd.DataFrame(mse(X_train, pred_x_train), columns=['Error'])
test_mae_loss = pd.DataFrame(mse(X_test, pred_x_test), columns=['Error'])

### Selection Threshold

threshold = mean of the loss + 2 standard devation of the loss

In [110]:
test_mae_loss_array = test_mae_loss.to_numpy().flatten()

In [111]:
threshold = np.mean(test_mae_loss)[0] + 1.75 * np.std(test_mae_loss)[0]

print(threshold)

0.06595784123671382


In [112]:
test_mae_loss_array = test_mae_loss.to_numpy().flatten()

### Dates where anomalies have been detected

In [113]:
y_pred = test_mae_loss_array >= threshold
y_pred.shape

(4597,)

In [114]:
dates_test[:-(window_length-1)].shape

(4597,)

In [115]:
dates_with_hours_anomalies = dates_test[:-(window_length-1)][y_pred]

In [116]:
ref = dates_with_hours_anomalies
ref = pd.to_datetime(ref, format='%Y/%m/%d')

ref

DatetimeIndex(['2014-11-01 01:30:00', '2014-11-01 02:00:00',
               '2014-11-01 02:30:00', '2014-11-01 03:00:00',
               '2014-11-01 03:30:00', '2014-11-01 04:00:00',
               '2014-11-01 04:30:00', '2014-11-01 05:00:00',
               '2014-11-01 05:30:00', '2014-11-01 06:00:00',
               ...
               '2015-01-27 06:00:00', '2015-01-27 06:30:00',
               '2015-01-27 07:00:00', '2015-01-27 07:30:00',
               '2015-01-27 08:00:00', '2015-01-27 08:30:00',
               '2015-01-27 09:00:00', '2015-01-27 09:30:00',
               '2015-01-27 10:00:00', '2015-01-27 10:30:00'],
              dtype='datetime64[ns]', name='timestamp', length=182, freq=None)

In [117]:
lstm_data = data.reset_index().copy()

lstm_data['date'] = pd.to_datetime(lstm_data['timestamp']).dt.date

# Modifier les valeurs dans 'pred' où la date est dans 'ref'
lstm_data["pred"] = 0 
lstm_data.loc[lstm_data['date'].isin(ref.date), 'pred'] = 1

# Supprimer la colonne 'date' si elle n'est plus nécessaire
lstm_data = lstm_data.drop('date', axis=1)

lstm_data.to_csv("../data/lstm_nyc_taxi.csv")

In [118]:
total = pd.read_csv("../data/total_nyc_taxi.csv",index_col=0)
total["lstm_pred"] = lstm_data["pred"]

total.to_csv("../data/total_nyc_taxi.csv")